In [17]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import numpy as np
import sys
sys.path.append("..")
from pickle import dump
from dyntapy.supply_data import get_toy_network, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.visualization import show_network, show_demand
from dyntapy.assignments import StaticAssignment

In [39]:
import geopandas as gpd
import numpy as np
from sklearn.cluster import KMeans

# TODO make code more waterproof (naming of input and outputfile)
def aggregate_zones(city = "BRUSSEL", radius = "10-20", idealNbZonesPerCluster = 6):
    '''
    original shapefile will change: 'cluster' column containing the cluster to which each zone belongs
    new shapefill will be created: shapefile consisting of aggregated zones

    Note:
    code not waterproof: always looks in folder "STA_prep/shapefile_data/{city}_40_10/"
    '''
    inputFile = f"STA_prep/shapefile_data/{city}_40_10/{city}_40_10_{radius}.shp"
    outputFile = f"STA_prep/shapefile_data/{city}_40_10/{city}_40_10_{radius}_knn.shp"
    shapefile = gpd.read_file(inputFile)

    points = np.array(shapefile.centroid.apply(lambda p: [p.x, p.y]).tolist())

    k =  int(np.ceil(len(shapefile) / idealNbZonesPerCluster))  # number of clusters to create
    print(f'Aggregation of {len(shapefile)} zones into {k} clusters.')
    kmeans = KMeans(n_clusters=k, random_state=0).fit(points)

    shapefile["cluster"] = kmeans.labels_
    aggregated = shapefile.dissolve(by="cluster")
    basicStatisticsZoneSizes = shapefile.groupby('cluster').size().describe() # contains
    print(f"Basic statistics of zone aggregation:\n{basicStatisticsZoneSizes}\n")


    # Save the new shapefile to a file
    shapefile.to_file(inputFile)
    aggregated.to_file(outputFile)

aggregate_zones()
aggregate_zones(radius = "20-40", idealNbZonesPerCluster = 12)

Aggregation of 219 zones into 37 clusters.
Basic statistics of zone aggregation:
count    37.000000
mean      5.918919
std       2.408070
min       3.000000
25%       4.000000
50%       6.000000
75%       7.000000
max      12.000000
dtype: float64

Aggregation of 957 zones into 160 clusters.
Basic statistics of zone aggregation:
count    160.000000
mean       5.981250
std        3.802468
min        1.000000
25%        4.000000
50%        5.000000
75%        7.250000
max       26.000000
dtype: float64

